# Solving Linear Systems of Equations using HHL

## Setup
Initialize the IBMQ account for access to IBM quantum hardware.

This assumes a `.env` file in the project root directory with the following content:
```
IBMQ_TOKEN=<IBM_TOKEN>
```

In [2]:
from dotenv import load_dotenv
from qiskit import IBMQ
import os
load_dotenv()

IBMQ.save_account(token=os.getenv("IBMQ_TOKEN"), overwrite=True)

/tmp/ipykernel_2648896/2691405560.py:6: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.save_account(token=os.getenv("IBMQ_TOKEN"), overwrite=True)
/tmp/ipykernel_2648896/2691405560.py:6: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.save_account(token=os.getenv("IBMQ_TOKEN"), overwrite=True)


## HHL Qiskit Implementation

### Initialization

Import necessary libraries and define constants:
* Import QuantumRegister and QuantumCircuit from Qiskit.
* Import numpy to perform mathematical operations.
* Define constants, such as the number of qubits, angle θ for `|b⟩ = [cos(θ), sin(θ)]`, and elements of the matrix A.
   * In this implementation, `A` is a [tridiagonal symmetric matrix](https://en.wikipedia.org/wiki/Tridiagonal_matrix).

In [3]:
from qiskit import QuantumRegister, QuantumCircuit
import numpy as np

# Define constants
t = 2  # Time parameter for encoding the matrix A
NUM_QUBITS = 4  # Total number of qubits
nb = 1  # Number of qubits representing the solution
nl = 2  # Number of qubits representing the eigenvalues

theta = 0  # Angle defining |b>
a = 1  # Matrix diagonal
b = -1/3  # Matrix off-diagonal

Initialize quantum circuit and registers:
* Create a quantum register `qr` with the total number of qubits (`NUM_QUBITS`).
* Define the quantum register partitions (`qrb`, `qrl`, `qra`) for the solution, eigenvalue, and ancilla qubits respectively.
* Initialize a quantum circuit `qc` with the quantum registers `qr`.

In [4]:
# Initialize the quantum and classical registers
qr = QuantumRegister(NUM_QUBITS)

# Create a Quantum Circuit
qc = QuantumCircuit(qr)

# Define the quantum register partitions
qrb = qr[0:nb]  # Solution qubits
qrl = qr[nb:nb+nl]  # Eigenvalue qubits
qra = qr[nb+nl:nb+nl+1]  # Ancilla qubits

Apply a rotation (RY) gate on the first qubit (`qrb[0]`) with an angle of `2*θ` to prepare the state `|b⟩`.

In [5]:
# State preparation
qc.ry(2*theta, qrb[0])

### Quantum Phase Estimation

Performs QPE to estimate the eigenvalues of the matrix `A` encoded as a unitary operator `e^(iAt)`.

In [6]:
for qu in qrl:
    qc.h(qu)

qc.p(a*t, qrl[0])
qc.p(a*t*2, qrl[1])

qc.u(b*t, -np.pi/2, np.pi/2, qrb[0])

In [7]:
# Controlled e^{iAt} on \lambda_{1}:
params=b*t

qc.p(np.pi/2,qrb[0])
qc.cx(qrl[0],qrb[0])
qc.ry(params,qrb[0])
qc.cx(qrl[0],qrb[0])
qc.ry(-params,qrb[0])
qc.p(3*np.pi/2,qrb[0])

In [8]:
# Controlled e^{2iAt} on \lambda_{2}:
params = b*t*2

qc.p(np.pi/2,qrb[0])
qc.cx(qrl[1],qrb[0])
qc.ry(params,qrb[0])
qc.cx(qrl[1],qrb[0])
qc.ry(-params,qrb[0])
qc.p(3*np.pi/2,qrb[0])

### Inverse Quantum Fourier Transform

In [9]:
qc.h(qrl[1])
qc.rz(-np.pi/4,qrl[1])
qc.cx(qrl[0],qrl[1])
qc.rz(np.pi/4,qrl[1])
qc.cx(qrl[0],qrl[1])
qc.rz(-np.pi/4,qrl[0])
qc.h(qrl[0])

# TODO: Explain this step

In [10]:
# Eigenvalue rotation
t1=(-np.pi +np.pi/3 - 2*np.arcsin(1/3))/4
t2=(-np.pi -np.pi/3 + 2*np.arcsin(1/3))/4
t3=(np.pi -np.pi/3 - 2*np.arcsin(1/3))/4
t4=(np.pi +np.pi/3 + 2*np.arcsin(1/3))/4

qc.cx(qrl[1],qra[0])
qc.ry(t1,qra[0])
qc.cx(qrl[0],qra[0])
qc.ry(t2,qra[0])
qc.cx(qrl[1],qra[0])
qc.ry(t3,qra[0])
qc.cx(qrl[0],qra[0])
qc.ry(t4,qra[0])
qc.measure_all()

In [11]:
print(f"Depth: {qc.depth()}")
print(f"CNOTS: {qc.count_ops()['cx']}")
qc.draw(fold=-1)

Depth: 26
CNOTS: 10


┌───────┐┌──────────────────────┐┌────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌─────────┐┌────────┐┌───┐┌─────────────┐┌───┐┌────────────┐┌─────────┐                                                                                                               ░ ┌─┐         
  q0_0: ┤ Ry(0) ├┤ U(-0.66667,-π/2,π/2) ├┤ P(π/2) ├┤ X ├┤ Ry(-0.66667) ├┤ X ├┤ Ry(0.66667) ├┤ P(3π/2) ├┤ P(π/2) ├┤ X ├┤ Ry(-1.3333) ├┤ X ├┤ Ry(1.3333) ├┤ P(3π/2) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────░─┤M├─────────
        └─┬───┬─┘└───────┬──────┬───────┘└────────┘└─┬─┘└──────────────┘└─┬─┘└─────────────┘└─────────┘└────────┘└─┬─┘└─────────────┘└─┬─┘└────────────┘└─────────┘                      ┌──────────┐     ┌───┐                                                                   ░ └╥┘┌─┐      
  q0_1: ──┤ H ├──────────┤ P(2) ├────────────────────■────────────────────■────────────────────────────────────────┼───────────────────┼──────────────────────────────■───────────────■──┤ Rz(-π/4) ├─────┤ H ├────────■────────────────────────────────────────■─────────────────░──╫─┤M├──────
          ├───┤          ├──────┤                                                                                  │                   │      ┌───┐     ┌──────────┐┌─┴─┐┌─────────┐┌─┴─┐└──────────┘     └───┘        │                                        │                 ░  ║ └╥┘┌─┐   
  q0_2: ──┤ H ├──────────┤ P(4) ├──────────────────────────────────────────────────────────────────────────────────■───────────────────■──────┤ H ├─────┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├┤ X ├─────■────────────────────────┼────────────────────■───────────────────┼─────────────────░──╫──╫─┤M├───
          └───┘          └──────┘                                                                                                             └───┘     └──────────┘└───┘└─────────┘└───┘   ┌─┴─┐    ┌──────────────┐┌─┴─┐┌──────────────┐┌─┴─┐┌─────────────┐┌─┴─┐┌────────────┐ ░  ║  ║ └╥┘┌─┐
  q0_3: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Ry(-0.69352) ├┤ X ├┤ Ry(-0.87728) ├┤ X ├┤ Ry(0.35368) ├┤ X ├┤ Ry(1.2171) ├─░──╫──╫──╫─┤M├
                                                                                                                                                                                            └───┘    └──────────────┘└───┘└──────────────┘└───┘└─────────────┘└───┘└────────────┘ ░  ║  ║  ║ └╥┘
meas: 4/═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩══╩═
                                                                                                                                                                                                                                                                                     0  1  2  3

## Run on Quantum Hardware

Transpile the quantum circuit for running on IBM hardware.

In [12]:
from qiskit import IBMQ, transpile
from qiskit.utils.mitigation import complete_meas_cal

provider = IBMQ.load_account()

backend = provider.get_backend('ibmq_quito') # calibrate using real hardware
layout = [2,3,0,4]
chip_qubits = 5

# Transpiled circuit for the real hardware
qc_qa_cx = transpile(qc, backend=backend, initial_layout=layout)

#### Run with Error Mitigation Technique

In [13]:
meas_cals, state_labels = complete_meas_cal(qubit_list=layout,
                                            qr=QuantumRegister(chip_qubits))
qcs = meas_cals + [qc_qa_cx]

Run the job on the IBM quantum hardware.

In [14]:
job = backend.run(qcs, shots=10)

## I have no idea what is happening beyond this point. How do we get out a useful value? Maybe just avoid using the error mitigation because it is complicating things.

Use a fitter to reduce error using the error mitigation circuits created above.

In [37]:
from qiskit.utils.mitigation.fitters import CompleteMeasFitter

meas_fitter = CompleteMeasFitter(job.result(), state_labels)
mitigated_results = meas_fitter.filter.apply(job.result())

In [53]:
print(job.result().data(16))
print(mitigated_results.data(16))

{'counts': {'0x4': 1, '0x6': 2, '0x7': 1, '0xc': 1, '0xd': 5}}
{'counts': {'0001': 5.96552122971093e-16, '0010': 3.179347951287738e-16, '0011': 1.5829155982063684e-18, '0110': 2.8219785305853917, '0111': 0.8946071591611163, '1100': 0.31985346317598545, '1101': 5.963560847077506, '1110': 3.03462652471582e-16}}
